<a href="https://colab.research.google.com/github/tylaar1/PICAR-autopilot/blob/main/regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SWITCH TO **`T4 GPU`** OR THE **`HPC`**

# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

2025-03-11 16:47:44.021460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-11 16:47:44.034129: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-11 16:47:44.037973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 16:47:44.047276: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# makes it so pd dfs aren't truncated

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1) DATA PRE-PROCESSING

a) Load in labels + image file paths

b) combine them into one dataframe

c) EDA - spotted and removed erroneous label (speed = 1.42...)

- `cleaned_df` is the cleaned df with a) b) c) completed

d) convert images to numerical RGB feature maps - ML algorithms only understand numerical data

e) Splitting data into training and validation sets

f) data augmentation applied to training set

### 1a) load in labels + image file paths

In [ ]:
# labels_file_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_norm.csv' # tylers file path
labels_file_path = '/home/apyba3/KAGGLEDATAmachine-learning-in-science-ii-2025/training_norm.csv' # ben hpc file path (mlis2 cluster)
labels_df = pd.read_csv(labels_file_path, index_col='image_id')

In [ ]:
image_folder_path = '/home/apyba3/KAGGLEDATAmachine-learning-in-science-ii-2025//training_data/training_data' # ben hpc file path (mlis2 cluster)
# image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/training_data/training_data' # tylers file path
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'

Checking labels dataframe

In [ ]:
labels_df.head()

Checking image file paths dataframe - as you can see the file paths are ordered correctly (1.png, 2.png, 3.png, ...)

In [ ]:
imagefilepaths_df.head()

### 1b) Combine labels and image file paths into one dataframe

In [ ]:
merged_df = pd.merge(labels_df, imagefilepaths_df, on='image_id', how='inner')
merged_df['speed'] = merged_df['speed'].round(6) # to get rid of floating point errors

In [ ]:
merged_df.head()

In [ ]:
merged_df.loc[3139:3143]

The above cell shows that:

 1) the image files and labels match (see image_id and the number at the end of the file path)

 2) the missing rows in labels_df (image_id: 3141, 3999, 4895, 8285, 10171) have been taken care of

### 1c) EDA

In [ ]:
merged_df.value_counts('angle') #seems to favour right turn

note: imbalance datset

identifying the row with the erroneous speed value

In [ ]:
merged_df[merged_df['speed'] == 1.428571]

we want to remove this row

In [ ]:
cleaned_df = merged_df[merged_df['speed'] != 1.428571]
cleaned_df.loc[3882:3886]

### 1d) convert images to numerical RGB feature maps - make sure angle column is specified

In [ ]:
def process_image(image_path, label, resized_shape=(320, 240)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, resized_shape)
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image, label

dataset = tf.data.Dataset.from_tensor_slices((cleaned_df["image_file_paths"], cleaned_df["angle"])) # Convert pd df into a tf ds

dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.cache()
dataset = dataset.shuffle(len(cleaned_df))
dataset = dataset.batch(32)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

lets check and see if what we have done works

In [ ]:
for images, labels in dataset.take(1):
    print(images.shape, labels.shape)

### 1e) Splitting data into training and validation sets (test set is already provided in kaggle data)

In [ ]:
# 80-20 split

dataset_size = tf.data.experimental.cardinality(dataset).numpy()
train_size = int(0.8 * dataset_size)

train_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)

In [ ]:
print(f"Train size: {train_size}, validation size: {dataset_size - train_size}")

### 1f) Data augmentation applied to training set

Flipping or rotating the image will render the angle labels incorrect so none of that was applied to the images for this regression task

- Random Brightness Adjustment
- Random Contrast Adjustment
- Random Hue Adjustment
- Random Saturation Adjustment


In [ ]:
#only performing augmentation on training data as want validation/test data to reflect kaggle test set
seed = (6,9)
train_dataset = train_dataset.map(
      lambda image, label: (tf.image.stateless_random_brightness(image, 0.2,seed),label)
).map(
      lambda image, label: (tf.image.stateless_random_contrast(image,0.8,1.2,seed),label)
).map(
      lambda image, label: (tf.image.stateless_random_hue(image,0.2,seed),label)
).map(
      lambda image, label: (tf.image.stateless_random_saturation(image,0.8,1.2,seed),label)
).take(5
)

checking to see if whats been done was successful or needs debugging

In [ ]:
f, axarr = plt.subplots(1,10)

i = 0
for image_batch, label_batch in dataset.take(1):  # Take one batch
    for image in image_batch:  # Iterate through images in the batch
        if i < 10:  # Only display the first 5 images
            print('image shape: ', np.shape(image))
            tf.print('label:', label_batch[i])  # Print label for the corresponding image
            axarr[i].imshow(image)
            axarr[i].axis('off')
            i += 1
        else:
            break  # Stop after displaying 5 images
        plt.show()

# 2) Model Building - MobileNetV2 transfer learning

### 2a) MobileNetV2 transfer learning

In [ ]:
dropoutrate = 0.2
input_shape = (320, 240, 3)

mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

model = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='linear')
])
model.build()

mbnet.trainable = False

model.summary()

<ipython-input-19-c854c8fb879c>:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')


KeyboardInterrupt: 

In [ ]:
with tf.GradientTape() as tape:
  tape.reset() #this resets the gradient tape

In [ ]:
LR = 0.001 #learning rate
optimizer = tf.optimizers.Adam(LR) #adam optimiser

@tf.function
def train_step( model, X , Y):
    with tf.GradientTape() as tape:
        pred = model( X )
        Y = tf.cast(Y, tf.float32)
        current_loss = tf.reduce_mean(tf.losses.MeanSquaredError()( Y,  pred))
    grads = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients( zip( grads , model.trainable_variables) )
    current_MSE = tf.reduce_mean(tf.square(Y-pred))
    return(current_loss, current_MSE)

In [ ]:
niter = 20

tloss = []
tMSE = []
vloss = []
vMSE = []

for it in range(niter):
    for image_batch, label_batch in train_dataset:
      #for image, label in zip(image_batch, label_batch):
        #print(image)
        #print(label)
        #print(image.shape, label.shape)

        loss, MSE = train_step(model, image_batch, label_batch) #run training


    if it % 10 == 0: #log training metrics
      tf.print('iter: ',it, ', loss: {:.3f}, acc: {:.3f}'.format(loss, MSE))
      tloss.append(loss)
      tMSE.append(MSE)
'we should probably switch to balanced accuracy as eval method due to unbalanced data'
#commented out validation for now as it prints for each batch not each epoch massively slowwing process
'''
    if it % 50 == 0: #log validation metrics
      for val_image, val_label in validation_dataset:
        val_pred = model(val_image)
        val_int=tf.cast(val_label, dtype=tf.int32)
        val_loss = tf.reduce_mean(tf.losses.categorical_crossentropy(tf.one_hot(val_int,depth=2) , val_pred))
        val_acc = tf.reduce_mean(tf.metrics.categorical_accuracy(tf.one_hot(val_int,depth=2) , val_pred))
        tf.print('iter: ',it, ', validation loss: {:.3f}, validation acc: {:.3f}'.format(val_loss, val_acc))
        vloss.append(val_loss)
        vacc.append(val_acc)
'''

In [ ]:
'''
f, axarr = plt.subplots(1,10)

i = 0
for image_batch, label_batch in dataset.take(1):  # Take one batch
    for image in image_batch:  # Iterate through images in the batch
        if i < 10:  # Only display the first 5 images
            print('image shape: ', np.shape(image))
            tf.print('label:', label_batch[i])  # Print label for the corresponding image
            axarr[i].imshow(image)
            axarr[i].axis('off')
            i += 1
        else:
            break  # Stop after displaying 5 images
        plt.show()
'''

In [ ]:
model.save_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_frozen_regression.weights.h5')

In [ ]:
tf.keras.backend.clear_session() #Clear keras session

In [ ]:
dropoutrate = dropoutrate
input_shape = input_shape

mbnet =  tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

model = tf.keras.Sequential([
  mbnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(dropoutrate),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='linear')
])
model.build()

mbnet.trainable = False

model.summary()

In [ ]:
model.load_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_frozen_regression.weights.h5')

In [ ]:
mbnet.trainable = True
model.summary()

In [ ]:
LR = 0.0001
optimizer = tf.optimizers.Adam(LR) #adam optimiser


In [ ]:
niter = 20

tloss = []
tMSE = []
vloss = []
vMSE = []

for it in range(niter):
    for image_batch, label_batch in train_dataset:
      #for image, label in zip(image_batch, label_batch):
        #print(image)
        #print(label)
        #print(image.shape, label.shape)

        loss, MSE = train_step(model, image_batch, label_batch) #run training


    if it % 10 == 0: #log training metrics
      tf.print('iter: ',it, ', loss: {:.3f}, acc: {:.3f}'.format(loss, MSE))
      tloss.append(loss)
      tMSE.append(MSE)

In [ ]:
model.save_weights('/content/drive/MyDrive/machine-learning-in-science-ii-2025/car_unfrozen_regression.weights.h5')

In [ ]:
image_folder_path = '/content/drive/MyDrive/machine-learning-in-science-ii-2025/test_data/test_data'
image_file_paths = [
    os.path.join(image_folder_path, f)
    for f in os.listdir(image_folder_path)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

image_file_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0])) # sorts the files in the right order (1.png, 2.png, 3.png, ...)

imagefilepaths_df = pd.DataFrame(
    image_file_paths,
    columns=['image_file_paths'],
    index=[int(os.path.splitext(os.path.basename(path))[0]) for path in image_file_paths]
)

imagefilepaths_df.index.name = 'image_id'
imagefilepaths_df.head()

In [ ]:
def process_image_no_label(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Use decode_png for PNG images
    image = tf.image.resize(image, (128, 128))  # Resize to uniform shape
    image = image / 255.0  # Normalize pixel values to [0,1]
    return image
test_dataset = tf.data.Dataset.from_tensor_slices((imagefilepaths_df["image_file_paths"]))

test_dataset = test_dataset.map(process_image_no_label, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)


In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
predictions_df = pd.DataFrame(predictions)

In [ ]:
predictions_df.head()

In [ ]:
pd.DataFrame(predictions_df).to_csv('/content/drive/MyDrive/machine-learning-in-science-ii-2025/regressionpredictions.csv')

In [ ]:
predictions_df.value_counts()

# add data augmentation steps here

# transfer learning here